In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image

In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split

BATCH_SIZE = 32
IMAGE_SIZE = (320, 320)
dataset = "/content/drive/MyDrive/fire_dataset-2"

# Veri ön işleme ve dönüşümleri tanımlayın
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Veri setini yükleyin
full_dataset = datasets.ImageFolder(dataset, transform=transform)

# Veri setini train, validation ve test olarak bölün
train_size = int(0.8 * len(full_dataset))
valid_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split(full_dataset, [train_size, valid_size, test_size])

# DataLoader oluşturun
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [9]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)
model.fc = nn.Linear(512, 2)  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [12]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = train_loss / len(train_loader)

    return avg_loss, accuracy

def validate(model, valid_loader, criterion):
    model.eval()
    valid_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    avg_loss = valid_loss / len(valid_loader)

    return avg_loss, accuracy


In [13]:
epochs = 20

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    valid_loss, valid_acc = validate(model, valid_loader, criterion)

    print(f'Epoch: {epoch+1}/{epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.2f}%')


Epoch: 1/20, Train Loss: 1.7082, Train Acc: 73.09%, Valid Loss: 0.0550, Valid Acc: 95.96%
Epoch: 2/20, Train Loss: 0.0870, Train Acc: 97.12%, Valid Loss: 0.0732, Valid Acc: 96.97%
Epoch: 3/20, Train Loss: 0.0595, Train Acc: 98.12%, Valid Loss: 0.1130, Valid Acc: 95.96%
Epoch: 4/20, Train Loss: 0.0546, Train Acc: 98.87%, Valid Loss: 0.0559, Valid Acc: 97.98%
Epoch: 5/20, Train Loss: 0.0636, Train Acc: 98.12%, Valid Loss: 0.0669, Valid Acc: 96.97%
Epoch: 6/20, Train Loss: 0.0425, Train Acc: 98.75%, Valid Loss: 0.1330, Valid Acc: 95.96%
Epoch: 7/20, Train Loss: 0.0345, Train Acc: 99.50%, Valid Loss: 0.0654, Valid Acc: 96.97%
Epoch: 8/20, Train Loss: 0.0239, Train Acc: 99.50%, Valid Loss: 0.0454, Valid Acc: 96.97%
Epoch: 9/20, Train Loss: 0.0239, Train Acc: 99.75%, Valid Loss: 0.0659, Valid Acc: 95.96%
Epoch: 10/20, Train Loss: 0.0178, Train Acc: 99.75%, Valid Loss: 0.0492, Valid Acc: 96.97%
Epoch: 11/20, Train Loss: 0.0205, Train Acc: 99.37%, Valid Loss: 0.0440, Valid Acc: 97.98%
Epoch: 1

In [14]:
    def test(model, test_loader):
        model.eval()
        predictions = []
        true_labels = []

        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = outputs.max(1)

                predictions.extend(predicted.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions, average='weighted')
        recall = recall_score(true_labels, predictions, average='weighted')
        f1 = f1_score(true_labels, predictions, average='weighted')
        cm = confusion_matrix(true_labels, predictions)

        return accuracy, precision, recall, f1, cm

    test_acc, test_precision, test_recall, test_f1, test_cm = test(model, test_loader)

    print(f'Test Accuracy: {test_acc:.2f}%, '
        f'Precision: {test_precision:.2f}, '
        f'Recall: {test_recall:.2f}, '
        f'F1 Score: {test_f1:.2f}')
    print('Confusion Matrix:')
    print(test_cm)


Test Accuracy: 0.98%, Precision: 0.98, Recall: 0.98, F1 Score: 0.98
Confusion Matrix:
[[75  0]
 [ 2 24]]
